In [78]:
import pandas as pd

In [79]:
# 데이터 불러오기
train = pd.read_csv("../input/big-data-analytics-certification-kr-2022/train.csv")
test = pd.read_csv("../input/big-data-analytics-certification-kr-2022/test.csv")

In [80]:
train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,4
1,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,2
2,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,2
3,461319,Male,Yes,56,No,Artist,0.0,Average,2.0,Cat_6,3
4,460156,Male,No,32,Yes,Healthcare,1.0,Low,3.0,Cat_6,3


In [81]:
test.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,458989,Female,Yes,36,Yes,Engineer,0.0,Low,1.0,Cat_6
1,458994,Male,Yes,37,Yes,Healthcare,8.0,Average,4.0,Cat_6
2,459000,Male,Yes,59,No,Executive,11.0,High,2.0,Cat_6
3,459003,Male,Yes,47,Yes,Doctor,0.0,High,5.0,Cat_4
4,459005,Male,Yes,61,Yes,Doctor,5.0,Low,3.0,Cat_6


In [82]:
target = train.iloc[:,-1]
train = train.iloc[:,:-1]

In [84]:
train.dtypes == 'object'

ID                 False
Gender              True
Ever_Married        True
Age                False
Graduated           True
Profession          True
Work_Experience    False
Spending_Score      True
Family_Size        False
Var_1               True
dtype: bool

- 라벨 인코딩

In [85]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [86]:
for obj in train.dtypes.index:
    if train[obj].dtype == 'object':
        train[obj] = le.fit_transform(train[obj])
        test[obj] = le.transform(test[obj])

In [87]:
train = train.iloc[:,1:]
test = test.iloc[:,1:]

In [88]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           6665 non-null   int64  
 1   Ever_Married     6665 non-null   int64  
 2   Age              6665 non-null   int64  
 3   Graduated        6665 non-null   int64  
 4   Profession       6665 non-null   int64  
 5   Work_Experience  6665 non-null   float64
 6   Spending_Score   6665 non-null   int64  
 7   Family_Size      6665 non-null   float64
 8   Var_1            6665 non-null   int64  
dtypes: float64(2), int64(7)
memory usage: 468.8 KB


In [89]:
train.isnull().sum()

Gender             0
Ever_Married       0
Age                0
Graduated          0
Profession         0
Work_Experience    0
Spending_Score     0
Family_Size        0
Var_1              0
dtype: int64

- 학습 데이터

In [90]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train, target,
                                                 test_size=0.2,
                                                 random_state=42)

In [91]:
target

0       4
1       2
2       2
3       3
4       3
       ..
6660    2
6661    4
6662    4
6663    2
6664    2
Name: Segmentation, Length: 6665, dtype: int64

In [92]:
X_train.shape, X_val.shape, test.shape

((5332, 9), (1333, 9), (2154, 9))

- 모델 학습

In [107]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=300, max_depth=5)
model.fit(X_train, y_train)
pred = model.predict(test)
pred

array([1, 2, 2, ..., 1, 3, 4])

In [108]:
output = pd.DataFrame({'pred':pred})

In [109]:
output.to_csv('1234567.csv', index=False)

In [110]:
pd.read_csv('1234567.csv')

,pred
0,1
1,2
2,2
3,3
4,1
...,...
2149,1
2150,4
2151,1
2152,3


In [117]:
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report

In [112]:
pred2 = model.predict(X_val)

In [114]:
accuracy_score(y_val, pred2)

0.5191297824456114

In [116]:
print(confusion_matrix(y_val, pred2))

[[157  49  42  67]
 [ 86  77 110  29]
 [ 42  47 207  50]
 [ 89  17  13 251]]


In [118]:
print(classification_report(y_val, pred2, digits=3))

              precision    recall  f1-score   support

           1      0.420     0.498     0.456       315
           2      0.405     0.255     0.313       302
           3      0.556     0.598     0.577       346
           4      0.632     0.678     0.654       370

    accuracy                          0.519      1333
   macro avg      0.503     0.508     0.500      1333
weighted avg      0.511     0.519     0.510      1333

